# GPJax training benchmark

Used to know how long it takes to train a GPJax model on every ID of a benchmark dataset.

---

## Setup

In [1]:
import os
os.environ['JAX_ENABLE_X64'] = "True"

import time

import pandas as pd

In [2]:
dataset = "large"
shared_input = False
shared_hp = True

---

## Start timer

In [3]:
start = time.time()

---

## Data import

In [4]:
db = pd.read_csv(f"../dummy_datasets/{dataset}_{'shared_input' if shared_input else 'distinct_input'}_{'shared_hp' if shared_hp else 'distinct_hp'}.csv")
# db has 3 columns: ID, Input, Output

In [5]:
# First 90% of IDs are for training, last 10% for testing
train_ids = db["ID"].unique()[:int(0.9 * db["ID"].nunique())]
test_ids = db["ID"].unique()[int(0.9 * db["ID"].nunique()):]

db_train = db[db["ID"].isin(train_ids)]
db_test = db[db["ID"].isin(test_ids)]

# N.b: data is already sort by ID and Input in the toy datasets, but in a real case scenario, we would need to sort it

In [6]:
len(train_ids), len(test_ids)

(540, 60)

---

## Training

We train a GPJax model on every ID of the training set.

In [7]:
import jax
import jax.numpy as jnp
import gpjax as gpx
import optax
from jaxtyping import Float, Array

In [8]:
def train_gp_for_id(id_data):
	"""
	Train a GP for a single ID's data using GPJax - simple version
	"""
	# Convert to JAX arrays
	X = jnp.array(id_data['Input'].values, dtype=jnp.float64).reshape(-1, 1)
	y = jnp.array(id_data['Output'].values, dtype=jnp.float64).reshape(-1, 1)
	
	# Create dataset
	D = gpx.Dataset(X=X, y=y)
	
	# Define GP components
	meanf = gpx.mean_functions.Zero()
	kernel = gpx.kernels.RBF()
	prior = gpx.gps.Prior(mean_function=meanf, kernel=kernel)
	likelihood = gpx.likelihoods.Gaussian(num_datapoints=D.n)
	
	# Construct posterior
	posterior = prior * likelihood
	
	# Train the model
	opt_posterior, history = gpx.fit_scipy(
		model=posterior,
		objective=lambda p, d: -gpx.objectives.conjugate_mll(p, d),
		train_data=D,
	)
	
	return opt_posterior

In [9]:
# Train a GP for each ID in the training set
models = {}
training_times = []

print(f"Training GPs for {len(train_ids)} IDs with GPJax...")

for i, train_id in enumerate(train_ids):
	id_start_time = time.time()
	
	# Get data for this ID
	id_data = db_train[db_train["ID"] == train_id]
	
	# Train GP for this ID
	opt_posterior = train_gp_for_id(id_data)
	models[train_id] = opt_posterior
	
	id_time = time.time() - id_start_time
	training_times.append(id_time)
	
	# Print progress every 20 IDs
	if (i + 1) % 20 == 0 or i == len(train_ids) - 1:
		avg_time = sum(training_times) / len(training_times)
		print(f"Completed {i + 1}/{len(train_ids)} IDs. Avg time per GP: {avg_time:.3f}s")

total_training_time = sum(training_times)
print(f"\nTotal training time: {total_training_time:.2f}s")
print(f"Average time per GP: {total_training_time/len(train_ids):.3f}s")

Training GPs for 540 IDs with GPJax...


/opt/anaconda3/envs/MagmaClustPy/lib/python3.12/site-packages/scipy/optimize/_minimize.py:733: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)


         Current function value: nan
         Iterations: 3
         Function evaluations: 114
         Gradient evaluations: 114
Optimization terminated successfully.
         Current function value: 4047.619972
         Iterations: 28
         Function evaluations: 30
         Gradient evaluations: 30
Optimization terminated successfully.
         Current function value: 4036.668880
         Iterations: 20
         Function evaluations: 24
         Gradient evaluations: 24
Optimization terminated successfully.
         Current function value: 4041.590925
         Iterations: 25
         Function evaluations: 28
         Gradient evaluations: 28
Optimization terminated successfully.
         Current function value: 4043.675450
         Iterations: 24
         Function evaluations: 27
         Gradient evaluations: 27
Optimization terminated successfully.
         Current function value: 4046.793284
         Iterations: 28
         Function evaluations: 30
         Gradient evaluations

/opt/anaconda3/envs/MagmaClustPy/lib/python3.12/site-packages/scipy/optimize/_minimize.py:733: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)


         Current function value: nan
         Iterations: 8
         Function evaluations: 124
         Gradient evaluations: 124


...

/opt/anaconda3/envs/MagmaClustPy/lib/python3.12/site-packages/scipy/optimize/_minimize.py:733: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)


         Current function value: 4098.138994
         Iterations: 10
         Function evaluations: 129
         Gradient evaluations: 118
Optimization terminated successfully.
         Current function value: 4038.914622
         Iterations: 21
         Function evaluations: 25
         Gradient evaluations: 25


/opt/anaconda3/envs/MagmaClustPy/lib/python3.12/site-packages/scipy/optimize/_minimize.py:733: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)


         Current function value: 4059.938239
         Iterations: 9
         Function evaluations: 31
         Gradient evaluations: 20
         Current function value: 3215.024452
         Iterations: 65
         Function evaluations: 110
         Gradient evaluations: 100
Completed 540/540 IDs. Avg time per GP: 1.383s

Total training time: 746.77s
Average time per GP: 1.383s


/opt/anaconda3/envs/MagmaClustPy/lib/python3.12/site-packages/scipy/optimize/_minimize.py:733: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)


In [10]:
# Final timing results
end = time.time()
total_elapsed = end - start

print(f"\n" + "="*50)
print(f"BENCHMARK RESULTS - GPJax")
print(f"="*50)
print(f"Dataset: {dataset}")
print(f"Number of IDs trained: {len(train_ids)}")
print(f"Total elapsed time: {total_elapsed:.2f}s")
print(f"Training time: {total_training_time:.2f}s")
print(f"Setup/overhead time: {total_elapsed - total_training_time:.2f}s")
print(f"Average time per GP: {total_training_time/len(train_ids):.3f}s")
print(f"="*50)


BENCHMARK RESULTS - GPJax
Dataset: large
Number of IDs trained: 540
Total elapsed time: 749.62s
Training time: 746.77s
Setup/overhead time: 2.85s
Average time per GP: 1.383s
